# Résolution de systèmes linéaires

Nous considérons le système linéaire
$$
Ax = b
$$
où $A \in \mathbb{R}^{n \times n}$, $b \in \mathbb{R}^n$, et nous cherchons à déterminer le vecteur $x \in \mathbb{R}^n$. Nous supposons de plus que le rang de $A$ vaut $n$.

In [1]:
using LinearAlgebra
using BenchmarkTools

## Élimination de Gauss (algorithme naïf)

Adapté de https://lemesurierb.people.charleston.edu/introduction-to-numerical-methods-and-analysis-julia/docs/linear-equations-1-row-reduction.html

Considérons de système
$$
\begin{pmatrix}
4 & 2 & 7 \\ 3 & 5 & -6 \\ 1 & -3 & 2
\end{pmatrix}
x =
\begin{pmatrix}
2 \\ 3 \\ 4
\end{pmatrix}
$$
Nous pouvons définir $A$ et $b$ comme suit.

In [55]:
A = [4.0 2.0 7.0; 3.0 5.0 -6.0; 1.0 -3.0 2.0];
b = [2.0; 3.0; 4.0];

Nous pouvons résoudre le système directement en Julia comme suit:

In [56]:
x = A\b

3-element Vector{Float64}:
  1.8116883116883116
 -1.0324675324675323
 -0.45454545454545453

La stratégie de base de la réduction de ligne ou de l'élimination de Gauss se décrit comme suit:
- Choisir une équation et isoler une variable en l'éliminant de toutes les autres équations non encore considérées;
- Répétez, de manière récursive, sur les équations restantes pour éliminer progressivement les inconnues restantes de toutes les autres équations.

On obtient une équation finale à une seule inconnue, dont on peut déduire immédiatement la valeur, et en injectant cette valeur dans les autres équations, on peut répéter le processus pour déterminer toutes les variables, une à une.

La méthode la plus simple, qui cependant ne fonctionnera pas dans tous les cas, consiste à éliminer progressivement la $i^e$ variable de la $i^e$ contrainte, pour $i$ allant de 1 jusque $n$. Cette approche se résume dans le code suivant.

In [73]:
function rowreduce(A, b)
    # On copie A et b pour ne pas modifier la matrice et le vecteur d'origine.
    U = copy(A)
    c = copy(b)
    n = length(b)
    L = zeros(n, n)

    for k in 1:n-1
        for i in k+1:n
            # élimine la variable k des équations k+1 à n
            ℓ = U[i,k] / U[k,k]  # cela suppose U[k,k] différent de 0...
            U[i,k+1:n] -= ℓ * U[k,k+1:n]

            # On met des 0 sous la diagonale principale dans la colonne k de U pour pouvoir l'illustrer.
            U[i,1:k] .= 0.
            # Mise à jour du terme de droite.
            c[i] -= ℓ * c[k]
        end
    end
    
    return (U, c)
end;

In [74]:
(U, c) = rowreduce(A, b)

([4.0 2.0 7.0; 0.0 3.5 -11.25; 0.0 0.0 -11.0], [2.0, 1.5, 5.0])

Autrement dit, on obtient le système
\begin{align*}
4x_1 + 2x_2 + 7x_3 &= 2 \\
3.5x_2 -11.25x_3 &= 1.5 \\
11x_3 &= 5.0
\end{align*}
Il est alors facile de calculer récursivement $x_k$, pour $k$ allant de $n$ à $1$ en décroissant.

In [75]:
for k in n:1
    x[k] = (c[k]-U[k,k+1:n]*x[k+1:n])/U[k,k]
end

In [76]:
x

3-element Vector{Float64}:
  1.8116883116883116
 -1.0324675324675323
 -0.45454545454545453

En pratique, des méthodes plus robustes doivent être employées et il est tentant de travailler en inversant explicitement $A$.

## Temps de calul et inversion

Nous allons créer un matrice test qui nous servira à calculer les temps de calcul en utilisant l'inversion matricielle ou les techniques de factorisation. L'inversion prend $O(n^3)$ opérations, tandis que la factorisation requiert $O(n^2)$ opérations, où $n$ est l'ordre de $A$ et la dimension de $b$ et $x$.

In [2]:
n = 10000

10000

In [3]:
A = zeros(n,n)
for i = 1:n
    A[i,i] = 2.0
end
for i = 1:n-1
    A[i,i+1] = A[i+1,i] = -1.0
end

In [4]:
A

10000×10000 Matrix{Float64}:
  2.0  -1.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   2.0  -1.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0   2.0  -1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0   2.0  -1.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0   2.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0   2.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   

In [5]:
inv(A)

10000×10000 Matrix{Float64}:
 0.9999      0.9998      0.9997      …  0.00029997  0.00019998  9.999e-5
 0.9998      1.9996      1.9994         0.00059994  0.00039996  0.00019998
 0.9997      1.9994      2.9991         0.00089991  0.00059994  0.00029997
 0.9996      1.9992      2.9988         0.00119988  0.00079992  0.00039996
 0.9995      1.999       2.9985         0.00149985  0.0009999   0.00049995
 0.9994      1.9988      2.9982      …  0.00179982  0.00119988  0.00059994
 0.9993      1.9986      2.9979         0.00209979  0.00139986  0.00069993
 0.9992      1.9984      2.9976         0.00239976  0.00159984  0.00079992
 0.9991      1.9982      2.9973         0.00269973  0.00179982  0.00089991
 0.999       1.998       2.997          0.0029997   0.0019998   0.0009999
 0.9989      1.9978      2.9967      …  0.00329967  0.00219978  0.00109989
 0.9988      1.9976      2.9964         0.00359964  0.00239976  0.00119988
 0.9987      1.9974      2.9961         0.00389961  0.00259974  0.00129987

Nous pouvons déjà remarque que $A$ est creuse, alors que son inverse est dense.

Créons le membre de droite du système.

In [6]:
b = ones(n)

10000-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

Nous résolvons à présent le système avec les deux techniques.

In [7]:
x1 = inv(A)*b

10000-element Vector{Float64}:
  5000.000000045553
  9999.000000091106
 14997.000000136679
 19994.00000018221
 24990.000000227945
 29985.000000273358
 34979.000000318825
 39972.00000036442
 44964.0000004102
 49955.00000045589
 54945.00000050126
 59934.000000546715
 64922.00000059264
     ⋮
 59934.00000044705
 54945.00000040977
 49955.00000037255
 44964.000000335225
 39972.00000029784
 34979.00000026076
 29985.000000223507
 24990.000000186257
 19994.000000148913
 14997.000000111753
  9999.000000074459
  5000.000000037228

In [8]:
x2 = A\b

10000-element Vector{Float64}:
  5000.000000045416
  9999.000000090831
 14997.000000136246
 19994.000000181662
 24990.00000022708
 29985.000000272492
 34979.0000003179
 39972.00000036331
 44964.00000040873
 49955.00000045414
 54945.00000049956
 59934.00000054497
 64922.00000059038
     ⋮
 59934.00000044711
 54945.000000409855
 49955.000000372595
 44964.000000335334
 39972.00000029807
 34979.00000026081
 29985.000000223554
 24990.000000186294
 19994.000000149037
 14997.000000111777
  9999.000000074517
  5000.000000037258

Comparons la précision des résultats.

In [9]:
[norm(A*x1-b), norm(A*x2-b)]

2-element Vector{Float64}:
 6.56076869699033e-6
 1.202289059402361e-7

Nous voyons que la technique de factorisation est légèremement plus précise.

Comparons les temps d'exécution.

In [10]:
@benchmark inv(A)*b

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 40.594 s (0.00% GC) to evaluate,
 with a memory estimate of 767.98 MiB, over 8 allocations.

In [11]:
@benchmark A\b

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 6.898 s (0.00% GC) to evaluate,
 with a memory estimate of 763.09 MiB, over 6 allocations.

Nous allons à présent exploiter le caractère creux de $A$.

In [12]:
using SparseArrays

In [13]:
A2 = sparse(A)

10000×10000 SparseMatrixCSC{Float64, Int64} with 29998 stored entries:
⎡⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⎦

In [14]:
inv(A2)

LoadError: The inverse of a sparse matrix can often be dense and can cause the computer to run out of memory. If you are sure you have enough memory, please either convert your matrix to a dense matrix, e.g. by calling `Matrix` or if `A` can be factorized, use `\` on the dense identity matrix, e.g. `A \ Matrix{eltype(A)}(I, size(A)...)` restrictions of `\` on sparse lhs applies. Alternatively, `A\b` is generally preferable to `inv(A)*b`

Nous voyons que Julia détecte que l'opération d'inversion serait inefficace au niveau mémoire.

In [15]:
x3 = A2\b

10000-element Vector{Float64}:
  4999.999999895034
  9998.99999979007
 14996.999999685106
 19993.999999580148
 24989.999999475192
 29984.999999370233
 34978.99999926528
 39971.99999916034
 44963.99999905539
 49954.99999895044
 54944.999998845495
 59933.99999874056
 64921.999998635605
     ⋮
 59933.99999992769
 54944.999999933745
 49954.99999993978
 44963.999999945816
 39971.99999995184
 34978.99999995786
 29984.999999963875
 24989.999999969892
 19993.999999975913
 14996.999999981937
  9998.99999998796
  4999.999999993979

In [16]:
@benchmark A2\b

BenchmarkTools.Trial: 637 samples with 1 evaluation.
 Range (min … max):  3.342 ms … 130.481 ms  ┊ GC (min … max): 0.00% … 12.06%
 Time  (median):     7.252 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.832 ms ±   7.703 ms  ┊ GC (mean ± σ):  0.84% ±  0.82%

                            ▂▄▇████▆▅▄▃▁▁                      
  ▅▄▆▄▅▄▁█▇▄▅▆▄▁▄▁▇▁▄▆▁▅▄▇▆▆██████████████▆▅▁▆▇█▆▁▄▁▅▁█▇▅█▁▆▇ █
  3.34 ms      Histogram: log(frequency) by time      10.7 ms <

 Memory estimate: 3.97 MiB, allocs estimate: 61.

In [17]:
norm(A*x3-b)

1.9513977835679233e-7

La précision reste similaire au cas dense, mais le temps d'exécution est significativement plus faible.

## Précision des résultats

Nous allons modifier la diagonale de la matrice pour accentuer les résultats.

In [18]:
for i = 1:n
    A[i,i] = 10.0^(-i)
end

A

10000×10000 Matrix{Float64}:
  0.1  -1.0    0.0     0.0      0.0     …   0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   0.01  -1.0     0.0      0.0         0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0    0.001  -1.0      0.0         0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   -1.0     0.0001  -1.0         0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0    0.0    -1.0      1.0e-5      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0    0.0     0.0     -1.0     …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0    0.0     0.0      0.0         0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0    0.0     0.0      0.0         0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0    0.0     0.0      0.0         0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0    0.0     0.0      0.0         0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0    0.0     0.0      0.0     …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0    0.0     0.0      0.0         0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0    0.0     0.0      0.0

Nous calculons le système suivant les deux techniques.

In [19]:
x1 = inv(A)*b

10000-element Vector{Float64}:
 -0.010011011222556957
 -1.001001101122256
 -0.9999989997886655
  1.1021224671919905e-6
 -1.0001011223779408e-6
 -1.0000011021324682
 -0.9999999998999799
  1.0021324680781207e-6
 -1.0001010222560183e-10
 -1.0000010021324681
 -0.99999999999999
  1.0021224680772933e-6
 -1.0001010222459176e-14
  ⋮
  0.0
 -1.000001002122467
 -1.0
  1.0021224670780926e-6
  0.0
 -1.000001002122467
 -1.0
  1.0021224670780926e-6
  0.0
 -1.000001002122467
 -1.0
  1.0021224672585038e-6

In [20]:
x2 = A\b

10000-element Vector{Float64}:
 -0.010011011222557096
 -1.001001101122256
 -0.9999989997886655
  1.1021224673889631e-6
 -1.000101122235779e-6
 -1.0000011021324684
 -0.9999999998999799
  1.0021324683416987e-6
 -1.0001009222046846e-10
 -1.0000010021324683
 -0.99999999999999
  1.0021224683409713e-6
 -1.0000010021224683e-14
  ⋮
 -0.0
 -1.0000010021224672
 -1.0
  1.0021224672307483e-6
 -0.0
 -1.0000010021224672
 -1.0
  1.0021224672307483e-6
 -0.0
 -1.0000010021224672
 -1.0
  1.002122467226311e-6

Calculons aussi en utilisant une matrice creuse.

In [21]:
A2 = sparse(A)

10000×10000 SparseMatrixCSC{Float64, Int64} with 20321 stored entries:
⎡⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⎦

In [22]:
x3 = A2\b

10000-element Vector{Float64}:
 -0.010011011222556765
 -1.0010011011222557
 -0.9999989997886656
  1.102122467089502e-6
 -1.0001011222232862e-6
 -1.0000011021324682
 -0.9999999998999799
  1.0021324681166794e-6
 -1.000101133349705e-10
 -1.0000010021324681
 -0.99999999999999
  1.0021224681247649e-6
 -1.0101020223459274e-14
  ⋮
 -0.0
 -1.000001002122467
 -1.0
  1.0021224670087037e-6
 -0.0
 -1.000001002122467
 -1.0
  1.0021224670087037e-6
 -0.0
 -1.000001002122467
 -1.0
  1.0021224670087037e-6

In [23]:
[norm(A*x1-b) norm(A*x2-b) norm(A*x3-b)]

1×3 Matrix{Float64}:
 7.94644e-15  2.93737e-16  2.93737e-16

À nouveau, nous voyons que la technique d'inversion donne des résultats moins intéressants.

Le phénonème peut être observé même sur des matrices de petite dimension quand la matrice est presque singulière.

In [24]:
b = [1.0 ; 1.0]
M = [ 1.01 1+10^(-12) ; 1+10^(-12) 1.01 ]

2×2 Matrix{Float64}:
 1.01  1.0
 1.0   1.01

In [25]:
det(M)

0.020099999997999798

In [26]:
x1 = inv(M)*b

2-element Vector{Float64}:
 0.4975124378106983
 0.4975124378106983

In [27]:
x2 = M\b

2-element Vector{Float64}:
 0.49751243781069776
 0.4975124378106977

In [28]:
[ norm(M*x1-b) norm(M*x2-b) ]

1×2 Matrix{Float64}:
 1.57009e-15  0.0

L'inversion est un peu moins précise, mais acceptable. Considérons une situation encore plus proche de la singularité.

In [29]:
M = [ 1.0 1+10^(-8) ; 1+10^(-8) 1.0 ]

2×2 Matrix{Float64}:
 1.0  1.0
 1.0  1.0

In [30]:
det(M)

-1.9999999967428275e-8

In [31]:
x1 = inv(M)*b

2-element Vector{Float64}:
 0.5
 0.5

In [32]:
x2 = M\b

2-element Vector{Float64}:
 0.4999999977755576
 0.4999999972244424

In [33]:
[ norm(M*x1-b) norm(M*x2-b) ]

1×2 Matrix{Float64}:
 7.07107e-9  1.11022e-16

Ici, la précision est nettement meilleure avec la factorisation.

Remarquons en fait qu'inverser une matrice revient à résoudre un système linéaire pour chaque vecteur de la base canonique.

In [34]:
Minv = M\I

2×2 Matrix{Float64}:
 -5.0e7   5.0e7
  5.0e7  -5.0e7

In [35]:
x3 = Minv*b

2-element Vector{Float64}:
 0.5
 0.5

In [36]:
norm(M*x3-b)

7.071067768891266e-9

On retrouve la même précision qu'avec l'utilisation de la fonction `inv`. En fait, les matrices sont les mêmes!

In [37]:
Minv-inv(M)

2×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0

Utiliser l'inversion revient donc à effectuer une étape coûteuse de calcul préalable, et à accumuler davantage les erreurs de calcul. De plus, si $A$ est creuse, $A^{-1}$ peut être dense et entraîner des problèmes mémoire.